In [ ]:
import numpy as np
import pandas as pd
import gurobipy as gp
from gurobipy import GRB, quicksum


class Production_data:
    def __init__(self, number_of_periods, number_of_items, demand_forecast, production_cost, holding_cost, setup_cost, item_requirements , capacity):
        self.T=number_of_periods
        self.items=number_of_items
        self.demand_forecast=np.array(demand_forecast)
        self.production_cost=np.array(production_cost)
        self.holding_cost=np.array(holding_cost)
        self.setup_cost=np.array(setup_cost)
        self.item_requirements=np.array(item_requirements)
        self.capacity=np.array(capacity)

file_name = "CLSP+ST-instances Data-R.xlsx"
#file_name = "prova2.xlsx"

xls = pd.ExcelFile(file_name)  # Read the whole file

tables_keywords = ["Demand Forecast:", "Production Cost", "Holding Cost", "Setup Cost", "UnitsOfCapacity", "Capacity"]


In [ ]:

def read_data(xls, sheet_name):
    tables_dict = {}
    df = pd.read_excel(xls, sheet_name=sheet_name)
    
    # Define which column to check for each keyword
    columns_to_check = {
        "Demand Forecast:": 0,
        "Production Cost": 0,
        "Holding Cost": 0,
        "Setup Cost": 0,
        "UnitsOfCapacity": 1,  # Check the second column for this keyword (it also contains setup time(i,2))
        "Capacity": 0
    }
    
    # Iterate through the keywords to find each table
    for keyword in tables_keywords:
        column_idx = columns_to_check.get(keyword, 0)
        
        # Check in the specified column for the keyword
        match = df[df.iloc[:, column_idx].astype(str).str.contains(keyword, na=False)]
        
        if not match.empty:
            table_start_row = match.index[0] + 1
            
            # Find the end of the current table (next keyword or empty rows)
            end_row = None
            for next_keyword in tables_keywords:
                if next_keyword != keyword:
                    next_column_idx = columns_to_check.get(next_keyword, 0)
                    next_match = df.loc[table_start_row:][df.loc[table_start_row:].iloc[:, next_column_idx].astype(str).str.contains(next_keyword, na=False)]
                    if not next_match.empty:
                        potential_end = next_match.index[0]
                        if end_row is None or potential_end < end_row:
                            end_row = potential_end
            
            # If no next keyword found, look for empty rows
            if end_row is None:
                for i in range(table_start_row, len(df)):
                    # Check if row is empty or contains only NaN values
                    if df.iloc[i].isna().all():
                        end_row = i
                        break
            
            # If still no end found, use the end of the dataframe
            if end_row is None:
                end_row = len(df)
            
            # Extract the table
            table_df = df.iloc[table_start_row:end_row]
            
            # Remove completely empty rows
            table_df = table_df.dropna(how='all')
            
            # Remove completely empty columns
            table_df = table_df.dropna(axis=1, how='all')
            
            # Remove any remaining NaN values by filling with 0
            table_df = table_df.fillna(0)
            
            # Convert to numpy array
            table = table_df.to_numpy()
            tables_dict[keyword] = table
    
    # Create an instance of Production_data class
    production_data = Production_data(
        tables_dict.get("Demand Forecast:", np.zeros((1,1))).shape[1]-1,
        tables_dict.get("Demand Forecast:", np.zeros((1,1))).shape[0]-1,  
        tables_dict.get("Demand Forecast:", np.zeros((1,1))),  
        tables_dict.get("Production Cost", np.zeros((1,1))),  
        tables_dict.get("Holding Cost", np.zeros((1,1))),  
        tables_dict.get("Setup Cost", np.zeros((1,1))),  
        tables_dict.get("UnitsOfCapacity", np.zeros((1,1))),  
        tables_dict.get("Capacity", np.zeros((1,1)))

    )
    
    production_data.capacity = np.vstack([
    np.zeros((1, production_data.capacity.shape[1]), dtype=production_data.capacity.dtype),production_data.capacity])

    production_data.item_requirements = np.vstack([
    np.zeros((1, production_data.item_requirements.shape[1]), dtype=production_data.item_requirements.dtype),production_data.item_requirements
])
    return production_data 

#data=read_data(xls, "Data-20-12 (1)")

#print(data.T)
#print(data.items)

#print(data.demand_forecast)
#print(data.holding_cost)
#print(data.setup_cost)
#print(data.capacity)
#print(data.production_cost)

#print(data.capacity[0, 1])
#print(data.capacity[12, 1])
#print(data.item_requirements)

sheet_list=["Data-20-12 (2)"]#, "Data-20-12 (2)", "Data-20-24 (1)", "Data-20-24 (2)" , "Data-100-24 (1)", "Data-100-24 (2)",
          #  "Data-200-24" ]

#sheet_list=["Sheet1"]





In [ ]:
import pandas as pd
import gurobipy as gp
from gurobipy import GRB, quicksum

def solve_MILP(data, sheet_name, results_df):

    model=gp.Model("MiCLSP-ST")

    T=data.T
    O=data.items

    #DUMMY AT PERIOD 0?

    #decision variables, constraint 5 and 6 added here
    y=model.addVars(O+1, T+1, vtype=GRB.BINARY, name="y" ) #produce of not for this item in this period
    s=model.addVars(O+1, T+1, vtype=GRB.CONTINUOUS, lb=0, name="s") #amount of inv of item i in period t
    x=model.addVars(O+1, T+1, vtype=GRB.CONTINUOUS, lb=0, name="x") #amount produced of item i in period t

    model.setObjective(
        quicksum(quicksum( data.setup_cost[i, t] * y[i, t] for i in range(1, O+1) ) for t in range(1, T+1))+
        quicksum(quicksum( data.production_cost[i, t] * x[i, t] for i in range(1, O+1) ) for t in range(1, T+1))+
        quicksum(quicksum( data.holding_cost[i, t] * s[i, t] for i in range(1, O+1) ) for t in range(1, T+1)),
        GRB.MINIMIZE
    )  


    #constraint 2
    for t in range(1, T+1):
        for i in range(1, O+1):
            model.addConstr(s[i, t-1]+x[i, t]-s[i, t]==data.demand_forecast[i, t])

    

    #constraint 3, this definitely works
    for t in range(1, T+1):
        print(data.capacity[t,1])
        model.addConstr( quicksum( x[i, t]*data.item_requirements[i, 1] + data.item_requirements[i,2] * y[i,t] for i in range(1, O+1))<= data.capacity[t, 1])
        
    #constraint 4, this defintely ok
    for t in range(1, T+1):
        for i in range(1, O+1):
            model.addConstr(x[i, t]-(quicksum(data.demand_forecast[i, q] for q in range(t, T+1))) * y[i, t]<=0)

    #constraint 7, no initial inventory for all items, this definitely works
    for i in range(1, O+1):
        model.addConstr( s[i, 0] == 0, name="no init inv") #lloks like this constrint is ok
    
    #solve the model
    model.optimize()

    if model.Status == GRB.OPTIMAL:
        print("\noptimal solution found:")
        print(model.ObjVal)

        # Create a list to store all periods' results for this sheet
        period_results = []
        
        # Collect results for each period
        for t in range(1, T+1):
            period_results.append({
                "Sheet": sheet_name,
                "Period": t,
                "Inventory of item 1": s[3, t].X,   # hard-coded for item 1 only
                "Produced for item 1": x[3, t].X
            })
        
        # Add all periods at once to the results DataFrame
        sheet_results = pd.DataFrame(period_results)
        results_df = pd.concat([results_df, sheet_results], ignore_index=True)
    
    return results_df

In [ ]:
# Initialize results DataFrame with all needed columns
results_df = pd.DataFrame(columns=["Sheet", "Period", "Inventory of item 1", "Produced for item 1"])

for sheet_name in sheet_list:
    data = read_data(xls, sheet_name=sheet_name)
    results_df = solve_MILP(data, sheet_name, results_df)

results_df.to_csv("results_MILP.csv", index=False)

### Plant location reformulation

As for the single-item lot-sizing problem, an alternative formulation for the current problem, also 
called plant location reformulation (PLRFIP), is obtained by replacing in the model (MiCLSP-STIP) the 
production variables 𝑥𝑖𝑡 by the variables 𝑤𝑠𝑡 𝑖 given by:

Where 𝑤𝑠𝑡 𝑖 can be interpreted as the portion of demand of item 𝑖 ∈ 𝑃 in period 𝑡 ∈ 𝐻 fulfilled by 
production in period 𝑠 ∈ 𝐻,𝑠 ≤ 𝑡. Write down this reformulation and check the validity of your 
proposed model. 

In [ ]:
def solve_PLRF(data, sheet_name, results_df):

    model=gp.Model("PLRF")


    #decisions variables

    T=data.T
    O=data.items

    w=model.addVars(O+1, T+1, T+1, vtype=GRB.CONTINUOUS, lb=0, ub=1, name = "w" )#fraction of demand of item i produced in period s to satisfy period t
    y=model.addVars(O+1, T+1, vtype=GRB.BINARY, name="y") #produce or not for item i in period i

    model.setObjective(
        #setup cost
        quicksum(quicksum( data.setup_cost[i, t] * y[i, t] for i in range(1, O+1) ) for t in range(1, T+1))+
        #production cost, j is actual period and t in future period
        quicksum(quicksum(quicksum(data.production_cost[i, s] * data.demand_forecast[i,t] * w[i, s, t] 
                          for t in range(s, T+1)) # Changed range
                 for s in range(1, T+1))
        for i in range(1, O+1))+
        #holding cost, i per item
        quicksum( quicksum(quicksum
                           (sum(data.holding_cost[ i, t] for t in range(s, j)) * data.demand_forecast[i, j] * w[i, s, j] for j in range(s+1, T+1))
                           for s in range(1, T+1)) for i in range(1, O+1)),
        GRB.MINIMIZE
    )
    
    

    
    # Each demand must be fully satisfied (like each customer must be served) for each item
    for i in range(1, O+1):
        for t in range(1, T+1):
            model.addConstr( quicksum(w[i, s, t] for s in range(1, t+1)) == 1, name= f"demand_{i}{t}")
            
    #cannot produce any fraction of demand in period s is the variable y is 0, for item i
    for i in range(1, O+1):
        for t in range(1, T+1):
            for s in range(1, t+1):
                model.addConstr( w[i, s, t] <= y[i, s] )
        
    #still capacity constraints fossure
    #constraint 3

    for s in range(1, T+1):
        model.addConstr(
            quicksum(
                # Production time: sum over all demands being produced in period s
                sum(w[i, s, t] * data.demand_forecast[i, t] for t in range(s, T+1)) * data.item_requirements[i, 1] +
                # Setup time
                data.item_requirements[i, 2] * y[i, s]
                for i in range(1, O+1)
            ) <= data.capacity[s, 1],
            name=f"capacity_{s}"
        )
        
    
    #constraint 4
    for t in range(1, T+1):
        for i in range(1, O+1):
            model.addConstr( sum( w[i, t, s] for s in range(t, T+1) )-(quicksum(data.demand_forecast[i, q] for q in range(t, T+1))) * y[i, t]<=0)

    
    #solve
    model.optimize()

    if model.status == GRB.OPTIMAL:

        print("\noptimal solution found:")

        # Create a list to store all periods' results for this sheet
        period_results = []
        
        # Collect results for each period
        for t in range(1, T+1):
            period_results.append({
                "Sheet": sheet_name,
                "Period": t,
                "Setup Decision": y[1, t].X,
                "Production Quantity": sum(w[2, t, j].X * data.demand_forecast[2, j] for j in range(t, T+1)),
                "Capacity Usage": sum(sum(w[i, t, j].X * data.demand_forecast[i, j] * data.item_requirements[i, 1] + data.item_requirements[i, 2] * y[i,t].X  
                                        for j in range(t, T+1)) 
                                    for i in range(1, O+1)) / data.capacity[t, 1],
                "Total Cost": model.objVal
            })
        
        sheet_results = pd.DataFrame(period_results)
        results_df = pd.concat([ results_df, sheet_results ], ignore_index=False)


    return results_df


In [ ]:

# Initialize results DataFrame with all needed columns
results_df = pd.DataFrame(columns=["Sheet", "Period", "Produced in the period for item 1"])

for sheet_name in sheet_list:
    data = read_data(xls, sheet_name=sheet_name)
    results_df = solve_PLRF(data, sheet_name, results_df)

results_df.to_csv("results_PLRF.csv", index=False)

## Lagrangian relaxation of Constraint (3)

In [ ]:
import matplotlib.pyplot as plt
import gurobipy as gp 
from gurobipy import GRB
import pandas as pd
import numpy as np
import re

In [ ]:
def get_data(sheet_name):
    file_path = "CLSP+ST-instances Data-R.xlsx"  
    numbers = re.findall(r'\d+', sheet_name)
    number_items = int(numbers[0])
    number_capacity = int(numbers[1])
    df = pd.read_excel(file_path, sheet_name=sheet_name, header=None)

    # Identify matrix boundaries manually or by searching keywords
    demand_start = df[df[0] == "Demand Forecast:"].index[0] + 1
    production_start = df[df[0] == "Production Cost"].index[0] + 1
    holding_start = df[df[0] == "Holding Cost"].index[0] + 1
    setup_start = df[df[0] == "Setup Cost"].index[0] + 1
    capacity_start = df[df[0] == "Capacity"].index[0] + 1

    # Extract matrices and remove first row and first column
    demand_forecast = df.iloc[demand_start+1:production_start-2, 1:].reset_index(drop=True)
    production_cost = df.iloc[production_start+1:holding_start-2, 1:].reset_index(drop=True)
    holding_cost = df.iloc[holding_start+1:setup_start-2, 1:].reset_index(drop=True)
    setup_cost = df.iloc[setup_start+1:setup_start+number_items+1, 1:].reset_index(drop=True)
    capacity = df.iloc[capacity_start:capacity_start+number_capacity+1, 1:2].reset_index(drop=True)

    UnitsOfCapacity_SupTime = df.iloc[setup_start+number_items+2:capacity_start-2, 1:3].reset_index(drop=True)
    UnitsOfCapacity_SupTime.columns = UnitsOfCapacity_SupTime.iloc[0]
    UnitsOfCapacity_SupTime = UnitsOfCapacity_SupTime[1:].reset_index(drop=True)
    UnitsOfCapacity = UnitsOfCapacity_SupTime['UnitsOfCapacity']
    SetupTime = UnitsOfCapacity_SupTime['SetupTime']

    # Convert to numeric values
    demand_forecast = demand_forecast.apply(pd.to_numeric, errors='coerce')
    production_cost = production_cost.apply(pd.to_numeric, errors='coerce')
    holding_cost = holding_cost.apply(pd.to_numeric, errors='coerce')
    setup_cost = setup_cost.apply(pd.to_numeric, errors='coerce')

    data = {}
    data['Demand Forecast'] = demand_forecast
    data['Production Cost'] = production_cost
    data['Holding Cost'] = holding_cost
    data['Setup Cost'] = setup_cost
    data['UnitsOfCapacity'] = UnitsOfCapacity
    data['SetupTime'] = SetupTime
    data['Capacity'] = capacity
    
    return data

In [ ]:
data = get_data('Data-20-12 (1)')
data.keys()

In [ ]:
# Take constraint 3 as the hard constraint

model = gp.Model("LR")

N, T = data['Demand Forecast'].shape

d = np.array(data['Demand Forecast'])
c = np.array(data['Production Cost'])
h = np.array(data['Holding Cost'])
f = np.array(data['Setup Cost'])
r = np.array(data['UnitsOfCapacity'])
k = np.array(data['Capacity']).flatten()
tau = np.array(data['SetupTime'])

mu = np.ones(T)
dual_bounds = []
best_dual_bound = float('-inf')
norm = [np.linalg.norm(mu)]

iteration = 0
while iteration <= 2000:
    iteration += 1
    print(f'Iteration: {iteration}')
    
    y = model.addVars(N, T, vtype=GRB.BINARY, name="y")
    s = model.addVars(N, T, vtype=GRB.CONTINUOUS, lb=0, name="s")
    x = model.addVars(N, T, vtype=GRB.CONTINUOUS, lb=0, name="x")

    model.setObjective(
        gp.quicksum(
            gp.quicksum(
                f[i, t] * y[i, t] +
                c[i, t] * x[i, t] +
                h[i, t] * s[i, t] +
                mu[t] * (r[i] * x[i, t] + tau[i] * y[i, t])
                for i in range(N)
            ) for t in range(T)
        ) - gp.quicksum(mu[t] * k[t] for t in range(T)), 
        GRB.MINIMIZE
    )    

    for t in range(T):
        for i in range(N):
            if t == 0:
                model.addConstr(x[i, t] - s[i, t] == d[i, t])
            else:
                model.addConstr(s[i, t-1] + x[i, t] - s[i, t] == d[i, t])
                
            model.addConstr(x[i, t] - gp.quicksum(d[i, q] for q in range(t, T)) * y[i, t] <= 0)
    
#     for t in range(T):
#         model.addConstr(gp.quicksum(r[i]*x[i, t] + tau[i]*y[i, t] for i in range(N))  <= k[t])
    

    model.optimize()
    
    dual_obj = model.ObjVal
    dual_bounds.append(dual_obj)
    best_dual_bound = max(best_dual_bound, dual_obj)
    norm.append(np.linalg.norm(mu))
    
    subgradient = np.array([
        sum(r[i] * x[i, t].X + tau[i] * y[i, t].X for i in range(N)) - k[t]
        for t in range(T)
    ])
    
    if len(dual_bounds) > 100 and np.mean(dual_bounds[-10:]) - best_dual_bound < 0:
        break
        
    mu = np.maximum(0, mu + subgradient/np.max(np.abs(subgradient)*10))
    
    

## DANTZIG WOLF

# Benders' Decomposition

Benders' Decomposition is a method for solving large-scale optimization problems with a specific block structure. It decomposes the problem into a **Master Problem (MP)** and a **Subproblem (SP)**, iteratively solving them to find the optimal solution.

## Problem Decomposition

- **Master Problem (MP):** A relaxed version of the original problem, focusing on a subset of the variables (often integer variables).
- **Subproblem (SP):** A smaller problem obtained by fixing the variables of the Master Problem, often resulting in a linear program.

## Algorithm Steps

### **Initialization**
Solve the Master Problem to obtain an initial solution \( y_k \).

### **Iteration \( k \)**
1. **Solve the Subproblem:**
   - Fix the complicating variables \( y \) at the value \( y_k \) and solve the Subproblem.
   - If the Subproblem is **optimal**, obtain its objective value \( v(y_k) \) and the dual variables \( \pi^k \) associated with its constraints.
   - If the Subproblem is **infeasible**, obtain a feasibility cut using Farkas' Lemma, characterized by \( \alpha^k \) and \( \beta^k \).

2. **Generate Cut:**
   - **Optimality Cut:**  
     $$
     \theta \geq v(y_k) + \sum_i \sum_t \pi^k_t \cdot (h_{it}(y_{it} - y^k_{it}))
     $$
   - **Feasibility Cut** (If Subproblem is infeasible):  
     $$
     \sum_i \sum_t \alpha^k_t \cdot h_{it} \cdot y_{it} \leq \beta^k
     $$
   - Where:
     - \( h_{it} \) represents how \( y \) affects the constraints in the Subproblem.
     - \( \alpha^k_t \) and \( \beta^k \) are derived from Farkas' Lemma.

3. **Add Cut to Master Problem:**
   - The generated cut (optimality or feasibility) is added to the Master Problem to refine its solution space.

4. **Solve Master Problem:**
   - Solve the Master Problem, including the new cut, to obtain an updated solution \( y_{k+1} \) and a **lower bound (LB)** on the optimal solution.

5. **Update Upper Bound (UB):**
   - The upper bound (UB) is updated by taking the minimum of the current UB and the objective value of the Master Problem.

6. **Convergence:**
   - The algorithm terminates when the difference between the upper bound (UB) and the lower bound (LB) is less than or equal to a specified tolerance.

## Variables

- **y**: Complicating variables (variables in the Master Problem).  
- **x**: Variables in the Subproblem.  
- **theta**: Lower bound on the optimal value of the Subproblem.  
- **pi^k**: Dual variables obtained from solving the Subproblem at iteration \( k \).  
- **v(y_k)**: Optimal objective value of the Subproblem at iteration \( k \).  
- **(alpha^k,beta^k)**: Variables from Farkas' Lemma.  




In [ ]:
import numpy as np
import pandas as pd
import gurobipy as gp
from gurobipy import GRB, quicksum
import time

def solve_subproblem(data, T, O, demand, item_requirements, capacity_t, y_fixed):
    sub_model = gp.Model("Subproblem")
    x_sub = sub_model.addVars(O + 1, T + 1, vtype=GRB.CONTINUOUS, lb=0, name="x_sub")
    s_sub = sub_model.addVars(O + 1, T + 1, vtype=GRB.CONTINUOUS, lb=0, name="s_sub")
    pi = sub_model.addVars(T + 1, lb=-GRB.INFINITY, name="pi") # Dual variables for capacity constraints

    obj_sub = quicksum(quicksum(data.production_cost[i, t] * x_sub[i, t] for i in range(1, O + 1)) for t in range(1, T + 1))
    sub_model.setObjective(obj_sub, GRB.MINIMIZE)

    for t in range(1, T + 1):
        for i in range(1, O + 1):
            sub_model.addConstr(s_sub[i, t - 1] + x_sub[i, t] - s_sub[i, t] == demand[i, t], name=f"inventory_{i}_{t}")

    for t in range(1, T + 1):
        sub_model.addConstr(quicksum(x_sub[i, t] * item_requirements[i, 1] for i in range(1, O + 1)) <= capacity_t[t, 1] - quicksum(item_requirements[i, 2] * y_fixed[i, t] for i in range(1, O + 1)), name=f"capacity_{t}")

    for i in range(1, O + 1):
        sub_model.addConstr(s_sub[i, 0] == 0, name="no_init_inv_{i}")

    sub_model.Params.OutputFlag = 0
    sub_model.optimize()

    if sub_model.Status == GRB.OPTIMAL or sub_model.Status == GRB.SUBOPTIMAL:
        obj_val_sub = sub_model.ObjVal
        pi_values = {t: pi[t].X for t in range(1, T + 1)}
        return obj_val_sub, pi_values, True, None
    elif sub_model.Status == GRB.INFEASIBLE:
        # Get dual Farkas multipliers to generate a feasibility cut
        infeas_constr = sub_model.getConstrs()[-T:] # Capacity constraints are the ones that might cause infeasibility
        ray = sub_model.getAttr("FarkasDual", infeas_constr)
        return None, None, False, ray
    else:
        return None, None, False, None

def solve_master_problem(data, T, O, cuts):
    master_model = gp.Model("MasterProblem")
    y_master = master_model.addVars(O + 1, T + 1, vtype=GRB.BINARY, name="y_master")
    theta = master_model.addVar(lb=-GRB.INFINITY, name="theta")

    obj_master = quicksum(quicksum(data.setup_cost[i, t] * y_master[i, t] for i in range(1, O + 1)) for t in range(1, T + 1)) + theta
    master_model.setObjective(obj_master, GRB.MINIMIZE)

    for i, (cut_obj, cut_coeffs_y) in enumerate(cuts):
        master_model.addConstr(theta >= cut_obj + quicksum(quicksum(cut_coeffs_y[i, t] * (y_master[i, t] - 0) for i in range(1, O + 1)) for t in range(1, T + 1)), name=f"optimality_cut_{i}")

    master_model.Params.OutputFlag = 0
    master_model.optimize()

    if master_model.Status == GRB.OPTIMAL:
        y_master_values = {(i, t): y_master[i, t].X for i in range(1, O + 1) for t in range(1, T + 1)}
        lower_bound = master_model.ObjVal
        return y_master_values, lower_bound, True
    else:
        return None, None, False

def benders_decomposition(data, sheet_name, results_df, max_iterations=100, tolerance=1e-4):
    start_time = time.time()
    T = data.T
    O = data.items
    demand = data.demand_forecast
    item_requirements = data.item_requirements
    capacity = data.capacity

    y_current = {(i, t): 0 for i in range(1, O + 1) for t in range(1, T + 1)}
    lower_bound = -np.inf
    upper_bound = np.inf
    cuts = []
    iteration = 0

    while iteration < max_iterations and upper_bound - lower_bound > tolerance:
        iteration += 1
        print(f"\nIteration {iteration} - Sheet: {sheet_name}")
        print(f"Current Lower Bound: {lower_bound:.2f}, Current Upper Bound: {upper_bound:.2f}, Gap: {upper_bound - lower_bound:.2f}")

        sub_obj, pi_values, sub_optimal, infeas_ray = solve_subproblem(data, T, O, demand, item_requirements, capacity, y_current)

        if sub_optimal:
            setup_cost_current = sum(data.setup_cost[i, t] * y_current[i, t] for i in range(1, O + 1) for t in range(1, T + 1))
            current_upper_bound = setup_cost_current + sub_obj
            upper_bound = min(upper_bound, current_upper_bound)
            print(f"Subproblem Optimal. New Upper Bound: {upper_bound:.2f}")

            cut_objective = sub_obj - sum(capacity[t, 1] * pi_values[t] for t in range(1, T + 1))
            cut_coefficients_y = {}
            for i in range(1, O + 1):
                for t in range(1, T + 1):
                    cut_coefficients_y[i, t] = item_requirements[i, 2] * pi_values[t]
            cuts.append((cut_objective, cut_coefficients_y))

            y_next, next_lower_bound, master_optimal = solve_master_problem(data, T, O, cuts)
            if master_optimal:
                lower_bound = next_lower_bound
                print(f"Master Problem Solved. New Lower Bound: {lower_bound:.2f}, Number of Cuts: {len(cuts)}")
                y_current = y_next
            else:
                print("Master problem infeasible.")
                break

        elif infeas_ray is not None:
            print("Subproblem Infeasible. Generating feasibility cut.")
            # Formulate feasibility cut based on infeas_ray
            feasibility_cut_coeffs_y = {}
            feasibility_rhs = 0
            for t, ray_val in enumerate(infeas_ray, start=1):
                for i in range(1, O + 1):
                    feasibility_cut_coeffs_y[i, t] = item_requirements[i, 2] * ray_val
                    feasibility_rhs += capacity[t, 1] * ray_val # Should be <= 0

            # Need to formulate the cut properly to exclude y_current
            # This is a simplified placeholder; the actual form depends on the dual ray
            cuts.append((feasibility_rhs, feasibility_cut_coeffs_y))

            y_next, next_lower_bound, master_optimal = solve_master_problem(data, T, O, cuts)
            if master_optimal:
                lower_bound = next_lower_bound # Lower bound might not improve with feasibility cuts
                print(f"Master Problem Solved (after feasibility cut). Lower Bound: {lower_bound:.2f}, Number of Cuts: {len(cuts)}")
                y_current = y_next
            else:
                print("Master problem infeasible after feasibility cut.")
                break

        else:
            print("Subproblem status unknown.")
            break

    end_time = time.time()
    computation_time = end_time - start_time
    print(f"\nBenders' Decomposition finished in {computation_time:.2f} seconds.")
    print(f"Benders' Decomposition Solution Value (Z_BD): {upper_bound if upper_bound != np.inf else 'Not converged'}")
    print(f"Final Gap: {upper_bound - lower_bound if upper_bound != np.inf and lower_bound != -np.inf else 'Not converged'}")

    results_df = pd.concat([results_df, pd.DataFrame([{'Sheet': sheet_name, 'Z_BD': upper_bound if upper_bound != np.inf else None, 'Time_BD': computation_time, 'Gap_BD': upper_bound - lower_bound if upper_bound != np.inf and lower_bound != -np.inf else None, 'Iterations': iteration, 'Cuts': len(cuts)}])], ignore_index=True)

    return results_df

In [ ]:
sheet_list=["Data-20-12 (2)"]

# Initialize results DataFrame for Benders'
results_benders_df = pd.DataFrame(columns=["Sheet", "Z_BD", "Time_BD", "Gap_BD"])

for sheet_name in sheet_list:
    data = read_data(xls, sheet_name=sheet_name)
    results_benders_df = benders_decomposition(data, sheet_name, results_benders_df)

results_benders_df.to_csv("results_Benders.csv", index=False)

print("\nBenders' Decomposition results saved to results_Benders.csv")

## CHALLENGE

Reconsider the 6-periods capacitated single-item uncapacitated lot-sizing problem, and assume now 
that the demand in each period is normally distributed and follows the distribution N(mean = 100, sigma = 20). 
To approximately solve the recourse problem, we first start by discretizing the distribution of the 
random demand 𝒅𝒕 at the end of each period 𝑡. We assume that 𝒅𝒕 takes the realizations (mean ± 𝒌*sigma) 
for 𝑘 = 0,1.5,2.5! Approximate the probability corresponding to each realization.



In [ ]:
#so at each period we know that with a certain prob mean+-k*signma will happen

import scipy.stats as stats

mean=100
sigma=20
k=[0, 1.5, 2.5]

i=1
j=1
demand=[0] * 6
while i < 6:
    if i<4:
        #print(i)
        demand[i] = mean - k[i-1] * sigma
        #print(demand[i])
    else:
        demand[i] = mean + k[j] * sigma
        #print(demand[i])
        j=j+1
    i=i+1


demand[1:5] = sorted(demand[1:5])

for i in range (1, 6):
    print(demand[i])


print("\n")

data_points=[0]*6

for i in range (1, 6):
    data_points[i] = ( demand[i] - mean)/sigma

data_points[0:4] = sorted(data_points[1:5])

for i in range (0, 5):
    print(data_points[i])


prob = [0] * 5

# Compute probabilities
probabilities = [stats.norm.cdf( data_points[i], loc=mean, scale=sigma) for i in range (0, 5)]

for i in range(0,5):
    print(probabilities[i])


interval_probs = [probabilities[i] - probabilities[i-1] for i in range(1, len(probabilities))]
interval_probs.insert(0, probabilities[0])  # First interval probability

# Print results
print("Data Points:", data_points)
print("Probabilities:", interval_probs)




